<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/modeling_NiN/test_sample_NiN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ! rm -rf PyTorch-Architectures/
! git clone https://github.com/vishal-burman/PyTorch-Architectures.git

In [3]:
%cd PyTorch-Architectures/modeling_NiN/

/content/PyTorch-Architectures/modeling_NiN


In [4]:
import time

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader, Subset

from torchvision import datasets, transforms

from model import NiN

if torch.cuda.is_available():
  torch.backends.cudnn.deterministic = True

In [5]:
#############
# Settings
#############

RANDOM_SEED = 1
LEARNING_RATE = 0.0005
BATCH_SIZE = 256
EPOCHS = 10

# Architecture
NUM_CLASSES = 10

# Other
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
GRAYSCALE = False

In [6]:
########################
# CIFAR-10
########################

train_indices = torch.arange(0, 49000)
valid_indices = torch.arange(49000, 50000)

train_and_valid = datasets.CIFAR10(root='data',
                                   train=True,
                                   transform=transforms.ToTensor(),
                                   download=True)

train_dataset = Subset(train_and_valid, train_indices)
valid_dataset = Subset(train_and_valid, valid_indices)

test_dataset = datasets.CIFAR10(root='data',
                                train=False, 
                                transform=transforms.ToTensor())

########################
# DataLoaders
########################

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,
                          num_workers=4,
                          shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset,
                          batch_size=BATCH_SIZE,
                          num_workers=4,
                          shuffle=False)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=BATCH_SIZE,
                         num_workers=4,
                         shuffle=False)

############################
# Check dataset
############################
for images, labels in train_loader:
  print('Images Dimensions: ', images.shape)
  print('Labels Dimensions: ', labels.shape, "\n")
  break

for images, labels in valid_loader:
  print('Images Dimensions: ', images.shape)
  print('Labels Dimensions: ', labels.shape, "\n")
  break

for images, labels in test_loader:
  print('Images Dimensions: ', images.shape)
  print('Labels Dimensions: ', labels.shape, "\n")
  break

print('Length of train_loader: ', len(train_loader))
print('Length of valid_loader: ', len(valid_loader))
print('Length of test_loader: ', len(test_loader))

Extracting data/cifar-10-python.tar.gz to data
Images Dimensions:  torch.Size([256, 3, 32, 32])
Labels Dimensions:  torch.Size([256]) 

Images Dimensions:  torch.Size([256, 3, 32, 32])
Labels Dimensions:  torch.Size([256]) 

Images Dimensions:  torch.Size([256, 3, 32, 32])
Labels Dimensions:  torch.Size([256]) 

Length of train_loader:  192
Length of valid_loader:  4
Length of test_loader:  40


In [7]:
torch.manual_seed(RANDOM_SEED)

model = NiN(num_classes=NUM_CLASSES, is_memory_efficient=True)
model.to(DEVICE)

params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('Total trainable parameters: ', params)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

Total trainable parameters:  966986


In [8]:
def compute_accuracy(model, data_loader, device):
  correct_pred, num_examples = 0, 0
  for idx, (features, targets) in enumerate(data_loader):
    features = features.to(device)
    targets = targets.to(device)

    logits, probas = model(features)
    _, predicted_labels = torch.max(probas, 1)
    correct_pred += (predicted_labels == targets).sum()
    num_examples += targets.size(0)
  return correct_pred.float()/ num_examples * 100

start_time = time.time()
for epoch in range(EPOCHS):
  model.train()
  for idx, (features, targets) in enumerate(train_loader):
    features = features.to(DEVICE)
    targets = targets.to(DEVICE)

    logits, _ = model(features)

    loss = F.cross_entropy(logits, targets)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # LOGGING
    if idx % 120 == 0:
      print('Batch: %04d/%04d || Epoch: %04d/%04d || Loss: %.3f' % (idx, len(train_loader), epoch+1, EPOCHS, loss.item()))
  
  model.eval()
  with torch.set_grad_enabled(False):
    train_accuracy = compute_accuracy(model, train_loader, DEVICE)
    valid_accuracy = compute_accuracy(model, valid_loader, DEVICE)
    print('Train Accuracy: %.2f' % (train_accuracy))
    print('Valid Accuracy: %.2f' % (valid_accuracy))
  epoch_elapsed_time = (time.time() - start_time) / 60
  print('Epoch Elapsed Time: %.2f min' % (epoch_elapsed_time))
total_elapsed_time = (time.time() - start_time) / 60
print('Total Elapsed Time: %.2f min' % (total_elapsed_time))

Batch: 0000/0192 || Epoch: 0001/0010 || Loss: 2.305
Batch: 0120/0192 || Epoch: 0001/0010 || Loss: 1.988
Train Accuracy: 25.42
Valid Accuracy: 26.10
Epoch Elapsed Time: 0.56 min
Batch: 0000/0192 || Epoch: 0002/0010 || Loss: 1.920
Batch: 0120/0192 || Epoch: 0002/0010 || Loss: 1.727
Train Accuracy: 37.59
Valid Accuracy: 37.70
Epoch Elapsed Time: 1.14 min
Batch: 0000/0192 || Epoch: 0003/0010 || Loss: 1.644
Batch: 0120/0192 || Epoch: 0003/0010 || Loss: 1.710
Train Accuracy: 41.98
Valid Accuracy: 43.80
Epoch Elapsed Time: 1.72 min
Batch: 0000/0192 || Epoch: 0004/0010 || Loss: 1.532
Batch: 0120/0192 || Epoch: 0004/0010 || Loss: 1.503
Train Accuracy: 46.33
Valid Accuracy: 46.70
Epoch Elapsed Time: 2.30 min
Batch: 0000/0192 || Epoch: 0005/0010 || Loss: 1.417
Batch: 0120/0192 || Epoch: 0005/0010 || Loss: 1.490
Train Accuracy: 49.79
Valid Accuracy: 50.70
Epoch Elapsed Time: 2.88 min
Batch: 0000/0192 || Epoch: 0006/0010 || Loss: 1.476
Batch: 0120/0192 || Epoch: 0006/0010 || Loss: 1.396
Train Accur

In [9]:
! nvidia-smi --format=csv --query-gpu=memory.used

memory.used [MiB]
2695 MiB
